## Required Library Import

In [1]:
import ee
ee.Authenticate()
ee.Initialize()

EEException: Cannot authenticate: Invalid request.

In [3]:
import geemap
from epistemx.module_1 import Reflectance_Data, Reflectance_Stats
from epistemx.helpers import get_aoi_from_gaul

2025-10-03 10:15:37,982 - numexpr.utils - INFO - NumExpr defaulting to 12 threads.


## Image Collection Retrival

### Multispectral Bands

In [ ]:
# --- Area of Interest (AOI) ---
#Set the country and province for the AOI using GAUL admin boundaries
aoi = get_aoi_from_gaul(country="Indonesia", province="Sumatera Selatan")
#Alternatively, used geemap_shp_to_ee to directly used shapefile in your local machine
#Intialize the relfectance class data function
optical_reflectance = Reflectance_Data()
#define the start and end date for imagery collection
start = '2017-01-01'
end = '2017-12-31'
#get the image collection and corresponding statistics
landsat_data, meta = optical_reflectance.get_optical_data(aoi, start, end, optical_data='L8_SR', 
                                                           cloud_cover=40, compute_detailed_stats=False)
#visualization parameter
l8_sr_visparam = {
    'min': 0,
    'max': 0.4,
    'gamma': [0.95, 1.1, 1],
    'bands':['NIR', 'RED', 'GREEN']
}
#create mosaic between image collection, and clip based on AOI
mosaic_landsat = landsat_data.mosaic().clip(aoi)
#Alternatively you can use temporal aggregation (ee reducer) to create mode cloudless imagery
median_landsat = landsat_data.median().clip(aoi)
#Add the data to the map
Map = geemap.Map()
Map.addLayer(mosaic_landsat, l8_sr_visparam, 'L8 SR Mosaic')
Map.addLayer(median_landsat, l8_sr_visparam, 'L8 SR Median')
Map.addLayer(landsat_data, l8_sr_visparam, 'L8 SR Image Collection')
# set center of the map in the area of interest
Map.centerObject(aoi, 7)

2025-10-03 10:25:36,894 - Reflectance_Data - INFO - ReflectanceData initialized.
2025-10-03 10:25:36,895 - Reflectance_Data - INFO - Starting data fetch for Landsat 8 Operational Land Imager Surface Reflectance
2025-10-03 10:25:36,896 - Reflectance_Data - INFO - Date range: 2017-01-01 to 2017-12-31
2025-10-03 10:25:36,896 - Reflectance_Data - INFO - Cloud cover threshold: 40%
2025-10-03 10:25:36,897 - Reflectance_Data - INFO - detailed statistics will not be computed
2025-10-03 10:25:36,899 - Reflectance_Stats - INFO - Reflectance Stats initialized.
2025-10-03 10:25:36,899 - Reflectance_Data - INFO - Filtered collection created (use compute_detailed_stats=True for more information)


Map(center=[-3.2210694545062024, 104.16355582426586], controls=(WidgetControl(options=['position', 'transparen…

### Thermal Band

In [ ]:
#retive thermal bands from TOA
thermal_bands, thermal_stats = optical_reflectance.get_thermal_bands(aoi, start, end, cloud_cover=40, compute_detailed_stats=False)
median_thermal = thermal_bands.median().clip(aoi)
thermal_vis = {
    'min': 286,
    'max': 300,
    'gammma': 0.4
}
#stacked all landsat bands
stacked_landsat = median_landsat.addBands(median_thermal)
#visualize the thermal bands and multispectral bands
Map.addLayer(median_thermal, thermal_vis, "Thermal Bands")
Map

NameError: name 'optical_reflectance' is not defined

## Image Retrival Report

In [10]:
#intialize the statistic class
stats = Reflectance_Stats()
#get the retrival report and automatically print them
retrival_report = stats.get_collection_statistics(landsat_data, print_report=True)

2025-10-03 10:29:23,288 - Reflectance_Stats - INFO - Reflectance Stats initialized.


           Landsat Data Collection Retrival Report
Total Images Found: 53
Date Range: 2017-01-13 to 2017-12-17
Unique WRS Tiles: 10

Scene Cloud Cover Statistics:
------------------------------
Average Cloud Cover: 26.8%
Minimum Cloud Cover: 2.9%
Maximum Cloud Cover: 40.0%

WRS Path/Row Tiles:
------------------------------
Path 123/Row 062
Path 123/Row 063
Path 124/Row 061
Path 124/Row 062
Path 124/Row 063
Path 124/Row 064
Path 125/Row 061
Path 125/Row 062
Path 125/Row 063
Path 126/Row 062

Available Acqusition Date:
------------------------------
Date range: 2017-01-13 to 2017-12-17
(53 total acquisition dates)

Scene IDs (first 10):
------------------------------
• LC08_123062_20170405
• LC08_123062_20170421
• LC08_123062_20170726
• LC08_123062_20170912
• LC08_123062_20171014
• LC08_123062_20171217
• LC08_123063_20170710
• LC08_123063_20170827
• LC08_123063_20170912
• LC08_123063_20171030
... and 43 more scenes

